<a href="https://colab.research.google.com/github/stef4k/train-maintenance-data-mining/blob/main/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text classification


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
from collections import Counter
import ast


from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN, RandomOverSampler
from imblearn.combine import SMOTEENN, SMOTETomek
from gensim.models import Word2Vec
from scipy.optimize import minimize
from sklearn.base import TransformerMixin
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN, RandomOverSampler
from imblearn.combine import SMOTEENN, SMOTETomek
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from scipy.sparse import csr_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from xgboost import XGBClassifier
from tqdm import tqdm
from scipy.optimize import differential_evolution
import numpy as np
import pandas as pd
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")


Manually remove the first ';' from the first row in csv file

In [ ]:
df = pd.read_csv('sncb_data_challenge.csv', delimiter=';', index_col=0)
df.sample(2)

,incident_id,vehicles_sequence,events_sequence,seconds_to_incident_sequence,approx_lat,approx_lon,train_kph_sequence,dj_ac_state_sequence,dj_dc_state_sequence,incident_type
846,4602047,"[1051, 1051, 1051, 1051, 1051, 1051, 1051, 105...","[4394, 1566, 1570, 2744, 4168, 4140, 3986, 400...","[-14390, -14390, -14390, -5621, -5520, -5501, ...",50.649112,5.57423,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, False, False, F...",13
267,4445901,"[506, 506, 506, 506, 506, 506, 506, 506, 506, ...","[2674, 4394, 1566, 1570, 4396, 2742, 4148, 402...","[-14380, -14209, -14209, -14209, -14121, -6476...",50.462717,4.82072,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, True, Tru...",4


Now I will analyze the percentage of each event type appearing at least once in an event sequence:

We save in a list all event codes that appear in less than 85% of the event sequences:

In [ ]:
events_low_frequency = list(map(int, list(sorted_events_perc_df[sorted_events_perc_df.percentage<=85].event_type)))

## Text preprocessing
Before we start with text classification we need to clean the sequences of events. As seen one value of `events_sequence` contains commas and brackets even though it is a string

In [ ]:
df.events_sequence.iloc[0]

'[2744, 4004, 2852, 4110, 2854, 4396, 1132, 4140, 4148, 2708, 4026, 1032, 1082, 4152, 4030, 4018, 4168, 4156, 4394, 152, 2742, 4410, 4406, 4068, 4408, 4412, 4066, 2744, 4026, 4148, 4168, 4140, 3986, 2744, 4002, 2852, 4110, 2854, 4148, 2708, 4026, 4140, 4152, 4030, 4018, 4140, 4168, 4156, 2852, 2854, 4124, 2858, 2658, 2688, 3254, 3254, 3254, 2970, 4082, 4090, 4092, 2982, 3236, 4100, 2702, 4394, 1250, 2970, 2980, 2970, 2980, 2970, 2982, 2970, 2982, 4168, 4140, 3986, 2742, 4004, 2852, 4110, 2854, 2982, 2708, 4026, 4030, 4018, 4148, 4140, 4152, 4168, 4156, 4120, 2858, 2658, 2688, 3254, 3254, 2970, 2982, 2708, 2970, 2982, 4100, 2702, 1250, 4394, 2744, 4026, 4148, 2970, 2980, 4168, 4140, 4168, 3986, 2744, 4002, 2852, 4110, 2854, 2980, 2708, 4026, 4148, 2552, 4168, 4140, 4152, 4030, 4018, 4026, 4140, 4168, 4156, 2970, 2982, 2708, 2970, 4082, 4092, 4090, 4084, 4094, 4090, 3236, 2982, 4100, 2702, 1250, 4394, 4168, 4140, 3986, 2744, 4004, 2852, 4110, 2854, 2982, 2708, 4026, 4140, 4030, 4018, 414

Also, as observed before some event types are so common they do not actually bring a lot of value (as mentioned in the paper as well). We remove those common event types

The steps to clean the event sequences are:
- keep non-common event types mentioned in list `events_low_frequency`
- remove symbols: [] , and store sequences of events as a string without brackets and commas:

In [42]:
df['clean_events_sequence'] = df.events_sequence.apply(ast.literal_eval).apply(lambda x: [i for i in x if i in events_low_frequency]).astype(str)\
                .replace(r'[\[\],]', '', regex=True)

  2%|▏         | 4/180 [4:04:58<179:39:05, 3674.69s/it]


## Text classification

Now we try to experiment using text techniques to transform the list events sequence:

In [ ]:
df['incident_type'].value_counts()
df = df[~df["incident_type"].isin([7, 16, 3, 6, 17])].copy()
df.reset_index(drop=True, inplace=True)

In [ ]:
target = df['incident_type'].copy() # target column separated
X_train, X_test, y_train, y_test = train_test_split(df.clean_events_sequence, target, test_size=0.2,  random_state=7)

Since the dataset is imbalanced we will use different strategies to battle that. Here we set a new sampling strategy based on a basic script:

In [ ]:
# Define custom sampling strategy based on class distribution
# Each non-majority class will have equal samples to 15% of the majority class plus their previous samples
class_counts = pd.Series(y_train).value_counts()
max_class_count = max(class_counts.values)
sampling_strategy = {class_counts.index[i]: int(max_class_count * 0.15) + class_counts.values[i]
                     for i in range(len(pd.Series(y_train).value_counts().index)) if class_counts.values[i] < max_class_count}
sampling_strategy

{99: 183, 14: 157, 9: 131, 2: 129, 4: 97, 11: 59}

Starting with CountVectorizer:
- Tokenization: Splits text into individual words (tokens).
- Builds a Vocabulary: Creates a dictionary of unique words (tokens) from the entire corpus.
- Counts the Occurrence: Calculates the frequency (count) of each word in each document.
- Transforms Text into a Sparse Matrix: Returns a matrix of shape (n_samples, n_features), where n_samples is the number of documents and n_features is the number of unique words in the vocabulary.

  We firstly set the sampling strategy for SMOTE:

Now we set the pipeline to be used:

In [ ]:
text_clf = Pipeline([
                    ('vect', CountVectorizer()),
                     #('decision_tree', DecisionTreeClassifier()),
                    ('smote', SMOTE(sampling_strategy=sampling_strategy, random_state=1, k_neighbors=2)),
                    ('extra_trees', ExtraTreesClassifier()),
                    #('random_forest', RandomForestClassifier())
                    ])

Training the model:

In [ ]:
text_clf.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()),
                ('smote',
                 SMOTE(k_neighbors=2, random_state=1,
                       sampling_strategy={2: 129, 4: 97, 9: 131, 11: 59,
                                          14: 157, 99: 183})),
                ('extra_trees', ExtraTreesClassifier())])

Print the results for the particular split of test data:

In [ ]:
clf_predict = text_clf.predict(X_test)
print(classification_report(y_test, clf_predict, zero_division=1))

              precision    recall  f1-score   support

           2       0.73      0.79      0.76        28
           4       0.83      0.79      0.81        19
           9       0.91      0.42      0.57        24
          11       1.00      0.00      0.00         5
          13       0.65      0.89      0.75        61
          14       0.70      0.53      0.60        30
          99       0.53      0.57      0.55        30

    accuracy                           0.68       197
   macro avg       0.76      0.57      0.58       197
weighted avg       0.71      0.68      0.66       197



In [ ]:
class Word2VecVectorizer(TransformerMixin):
    def __init__(self, size=100, window=5, min_count=1, workers=4):
        self.size = size
        self.window = window
        self.min_count = min_count
        self.workers = workers
        self.w2v_model = None

    def fit(self, X, y=None):
        sentences = [sentence.split() for sentence in X]
        self.w2v_model = Word2Vec(sentences, vector_size=self.size, window=self.window,
                                  min_count=self.min_count, workers=self.workers)
        return self

    def transform(self, X, y=None):
        transformed_data = np.array([
            np.mean([self.w2v_model.wv[word] for word in sentence.split() if word in self.w2v_model.wv]
                    or [np.zeros(self.size)], axis=0)
            for sentence in X
        ])
        return csr_matrix(transformed_data)

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X, y)

In [ ]:

class EnsembleModel:
    def __init__(self, trained_models, trained_vectorizers):

        self.trained_models = trained_models
        self.trained_vectorizers = trained_vectorizers
        self.optimized_weights = None

    def _generate_prediction_matrix(self, X):
        predictions = {}
        for (vect_name, samp_name, clf_name), model in self.trained_models.items():
            vectorizer = deepcopy(self.trained_vectorizers[vect_name])
            X_vect = vectorizer.transform(X).toarray()

            # Collect probability predictions for multiclass
            predictions[(vect_name, samp_name, clf_name)] = model.predict_proba(X_vect)

        # Convert predictions to a 3D array: (num_samples, num_classes, num_models)
        return np.stack(list(predictions.values()), axis=2)

    def fit(self, X_train, y_train):
        pred_matrix = self._generate_prediction_matrix(X_train)

        # Define fitness function for genetic algorithm
        def fitness(weights):
            # Apply weights to model predictions
            weighted_pred = np.tensordot(pred_matrix, weights, axes=([2], [0]))
            # Get the predicted class with the highest weighted probability
            final_pred = np.argmax(weighted_pred, axis=1)
            # Return negative weighted F1 score for optimization
            return -f1_score(y_train, final_pred, average='weighted')

        # Genetic algorithm for weight optimization
        num_models = pred_matrix.shape[2]
        bounds = [(0, 1)] * num_models
        result = differential_evolution(fitness, bounds)

        # Store optimized weights
        self.optimized_weights = result.x

    def predict(self, X):
        if self.optimized_weights is None:
            raise ValueError("The ensemble model must be trained using `train_ensemble` before prediction.")

        # Generate predictions for the input data
        pred_matrix = self._generate_prediction_matrix(X)
        weighted_pred = np.tensordot(pred_matrix, self.optimized_weights, axes=([2], [0]))
        ensemble_pred = np.argmax(weighted_pred, axis=1)
        return ensemble_pred


In [ ]:

class Representations:
  def __init__(self, df):
    self.df = df

  def representation_a(self, df):
            events_types_dict = {}
            for events_sequence in df['events_sequence']:
                row_list = ast.literal_eval(events_sequence)  # Transforming string into actual list
                unique_events = set(row_list)
                for event in unique_events:
                    if not events_types_dict.get(event):
                        events_types_dict[event] = 0
                    events_types_dict[event] += 1

            # Step 2: Sort events by frequency and calculate percentages
            sorted_dict = dict(sorted(events_types_dict.items(), key=lambda item: item[1], reverse=True))
            sorted_events_perc_df = pd.DataFrame(list(sorted_dict.items()), columns=['event_type', 'frequency'])
            sorted_events_perc_df['percentage'] = sorted_events_perc_df['frequency'] / df.shape[0] * 100
            sorted_events_perc_df['event_type'] = sorted_events_perc_df['event_type'].astype(str)  # Ensure event_type is string

            # Step 3: Filter events with percentage <= 85%
            events_low_frequency = list(map(int, list(sorted_events_perc_df[sorted_events_perc_df.percentage <= 85].event_type)))

            # Step 4: Clean the 'events_sequence' column
            df['clean_events_sequence'] = (
                df['events_sequence']
                .apply(ast.literal_eval)  # Convert string to list
                .apply(lambda x: [i for i in x if i in events_low_frequency])  # Filter low-frequency events
                .astype(str)
                .replace(r'[\[\],]', '', regex=True)  # Remove brackets and commas
            )

            # Step 5: Extract the target column
            self.y = df['incident_type'].copy()  # Separate target column
            self.X = df['clean_events_sequence'].copy()
            return self.X, self.y


In [73]:

class Experiment:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.le = LabelEncoder().fit(y)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=7, stratify=y)
        self.trained_models = {}
        self.trained_vectorizers = {}
        self.trainer_samplers = {}
        self.results = []
        self.sampling_strategies = {
            "SMOTE": SMOTE(sampling_strategy='auto', random_state=1, k_neighbors=3),
            #"Borderline-SMOTE": BorderlineSMOTE(sampling_strategy='auto', random_state=1, k_neighbors=3),
            #"ADASYN": ADASYN(sampling_strategy='auto', random_state=1, n_neighbors=3),
            #"RandomOversampler": RandomOverSampler(sampling_strategy='auto', random_state=1),
            #"SMOTE-ENN": SMOTEENN(sampling_strategy='auto', random_state=1),
            #"SMOTE-Tomek": SMOTETomek(sampling_strategy='auto', random_state=1)
        }

        self.vectorizers = {
            #"TFIDF": TfidfVectorizer(),
            "Count": CountVectorizer(),
            #"Word2Vec": Word2VecVectorizer(size=100, window=5, min_count=1)
        }
        self.classifiers = {
            'LogisticRegression': LogisticRegression(),
            'DecisionTree': DecisionTreeClassifier(),
            'RandomForest': RandomForestClassifier(),
            'ExtraTreesClassifier': ExtraTreesClassifier(),
            'GradientBoostingClassifier': GradientBoostingClassifier(),
            'AdaBoostClassifier': AdaBoostClassifier(),
            'GaussianNB': GaussianNB(),
            'KNN': KNeighborsClassifier(),
            'SVM': SVC(probability=True),
            'XGBoost': XGBClassifier(objective="multi:softmax", num_class=len(np.unique(y)), eval_metric="mlogloss", use_label_encoder=False),
        }


    def drop_small_classes(self, X, y, min_instances=25):
        # Convert y to a pandas Series if not already
        y_series = pd.Series(y) if not isinstance(y, pd.Series) else y

        # Count instances per class
        class_counts = y_series.value_counts()

        # Filter classes with at least min_instances
        valid_classes = class_counts[class_counts >= min_instances].index

        # Create a mask for rows with valid classes
        mask = y_series.isin(valid_classes)

        # Filter X and y
        X_filtered = X[mask] if isinstance(X, pd.DataFrame) else X[mask, :]
        y_filtered = y_series[mask]

        X_filtered.reset_index(drop=True, inplace=True)
        y_filtered.reset_index(drop=True, inplace=True)

        return X_filtered, y_filtered

    def test(self, model, model_name, vectorizer, vectorizer_name, sampler, sampler_name):
        """Test a model with Stratified K-Fold and return an array with metric results."""
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
        accuracies, recalls, precisions, f1s = [], [], [], []
        X, y = self.drop_small_classes(self.X, self.y)

        for train_index, test_index in skf.split(X, y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = self.le.transform((y[train_index])), self.le.transform(y[test_index])
            X_train = vectorizer.fit_transform(X_train).toarray()
            X_test = vectorizer.transform(X_test).toarray()

            X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)

            model.fit(X_resampled, y_resampled)

            y_pred = model.predict(X_test)

            # Collect metrics for each fold
            accuracies.append(accuracy_score(y_test, y_pred))
            recalls.append(recall_score(y_test, y_pred, average='weighted'))
            precisions.append(precision_score(y_test, y_pred, average='weighted'))
            f1s.append(f1_score(y_test, y_pred, average='weighted'))

        # Return average metrics as a result array
        return [
            model_name, vectorizer_name, sampler_name,
            np.mean(accuracies), np.std(accuracies),
            np.mean(recalls), np.std(recalls),
            np.mean(precisions), np.std(precisions),
            np.mean(f1s), np.std(f1s)
        ]

    def training(self):
        results = []
        progress_bar = tqdm(total = len(self.vectorizers.keys()) * len(self.sampling_strategies.keys()) * len(self.classifiers.keys()))
        # Iterate over vectorizers, samplers, and classifiers
        for vect_name, ovectorizer in self.vectorizers.items():
            vectorizer = deepcopy(ovectorizer)
            for samp_name, osampler in self.sampling_strategies.items():
                sampler = deepcopy(osampler)
                for clf_name, omodel in self.classifiers.items():
                    model = deepcopy(omodel)

                    result = self.test(
                        model=model,
                        model_name=clf_name,
                        vectorizer=vectorizer,
                        vectorizer_name=vect_name,
                        sampler=sampler,
                        sampler_name=samp_name
                    )
                    # Store the trained model and add result to the results list
                    results.append(result)

                    vectorizer = deepcopy(ovectorizer)
                    sampler = deepcopy(osampler)
                    model = deepcopy(omodel)
                    vectorizer.fit(self.X_train)
                    X_train = vectorizer.transform(self.X_train).toarray()
                    X_resampled, y_resampled = sampler.fit_resample(X_train, self.le.transform(self.y_train))
                    model.fit(X_resampled, y_resampled)
                    self.trained_models[(vect_name, samp_name, clf_name)] = model
                    self.trained_vectorizers[vect_name] = vectorizer
                    self.trainer_samplers[(vect_name, samp_name)] = sampler
                    progress_bar.update(1)

        # Convert results to a DataFrame for better readability
        columns = [
            'Model', 'Vectorizer', 'Sampler',
            'Accuracy Mean', 'Accuracy Std',
            'Recall Mean', 'Recall Std',
            'Precision Mean', 'Precision Std',
            'F1 Mean', 'F1 Std'
        ]
        results_df = pd.DataFrame(results, columns=columns)
        self.results = results_df
        return results_df




In [74]:
X, y = Representations(df).representation_a(df)
exp = Experiment(X, y)
results = exp.training()
ensemble = EnsembleModel(exp.trained_models, exp.trained_vectorizers)
ensemble.fit(exp.X_train, exp.y_train)
ensemble_predictions = ensemble.predict(exp.X_test)
print(classification_report(exp.y_test, ensemble_predictions, zero_division=1))

  0%|          | 0/10 [03:05<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

KeyError: 'key of type tuple not found and not a MultiIndex'

In [ ]:
ensemble = EnsembleModel(exp.trained_models, exp.trained_vectorizers)
ensemble.fit(exp.X_train, exp.y_train)
ensemble_predictions = ensemble.predict(exp.X_test)
print(classification_report(exp.y_test, ensemble_predictions, zero_division=1))

Now we calculate the cross validation:

In [ ]:
events_list = []
events_pre_incident = []
events_post_incident = []

for i, (events, seconds_to_incident_sequence, vehicles_sequence, train_kph_sequence, dj_ac_state_sequence, dj_dc_state_sequence) in tqdm(enumerate(zip(df["events_sequence"],
                                                                                                                                                      df["seconds_to_incident_sequence"],
                                                                                                                                                      df["vehicles_sequence"],
                                                                                                                                                      df["train_kph_sequence"],
                                                                                                                                                      df["dj_ac_state_sequence"],
                                                                                                                                                      df["dj_dc_state_sequence"])), total=len(df)):
    events = ast.literal_eval(events)
    seconds_to_incident_sequence = ast.literal_eval(seconds_to_incident_sequence)
    vehicles_sequence = ast.literal_eval(vehicles_sequence)
    train_kph_sequence = ast.literal_eval(train_kph_sequence)
    dj_ac_state_sequence = ast.literal_eval(dj_ac_state_sequence)
    dj_dc_state_sequence = ast.literal_eval(dj_dc_state_sequence)


    pre_incidents = []
    pre_incidents_vehicles = []
    pre_incidents_kph = []
    pre_incidents_ac = []
    pre_incidents_dc = []


    post_incidents = []
    post_incidents_vehicles = []
    post_incidents_kph = []
    post_incidents_ac = []
    post_incidents_dc = []

    event_seq = []
    prev_event = 0

    for event, time_to_incident, vehicle, kph, ac, dc in zip(events, seconds_to_incident_sequence, vehicles_sequence, train_kph_sequence, dj_ac_state_sequence, dj_dc_state_sequence):
      #if event != prev_event:
      event_seq.append(str(event))
      if time_to_incident <= 0:
          pre_incidents.append(str(event))
          pre_incidents_vehicles.append(str(vehicle))
          pre_incidents_kph.append(str(kph))
          pre_incidents_ac.append(str(ac))
          pre_incidents_dc.append(str(dc))
      else:
          post_incidents.append(str(event))
          post_incidents_vehicles.append(str(vehicle))
          post_incidents_kph.append(str(kph))
          post_incidents_ac.append(str(ac))
          post_incidents_dc.append(str(dc))
      #prev_event = event

    # Append the pre and post incident lists to the main lists
    events_pre_incident.append(pre_incidents)
    events_post_incident.append(post_incidents)
    events_list.append(event_seq)

100%|██████████| 1011/1011 [00:15<00:00, 64.88it/s] 
